# Clang ARM64 Minimal

2025-06-06

My personal notes on Clang AArch64

- Clang <https://clang.llvm.org/>
- JupyterLab <https://jupyter.org/>
- Termux <https://termux.dev/en/>
- Moto G4 with [Android 9.0_r44 ARM64 AOSiP](https://forum.xda-developers.com/t/rom-9-0_r44-arm64-android-open-source-illusion-project-athene-unofficial.3889942/)
- Processor: Snapdragon 617 octa-core ARMv8 Cortex-A53 64-bit

Some references:

- <https://stackoverflow.com/questions/200292/process-for-reducing-the-size-of-an-executable>
- <http://www.muppetlabs.com/~breadbox/software/tiny/>
- <https://stac47.github.io/c/relocation/elf/tutorial/2018/03/01/understanding-relocation-elf.html>
- <http://timelessname.com/elfbin/>
- <https://interrupt.memfault.com/blog/dealing-with-large-symbol-files>
- <https://www.reddit.com/r/C_Programming/comments/wdag9l/how_to_absolutely_minimize_the_executable/>
- <http://cs107e.github.io/guides/gcc/>

In [3]:
! jupyter-lab --version

4.4.3


In [4]:
! termux-info

Termux Variables:
TERMUX_APK_RELEASE=F_DROID
TERMUX_APP_PACKAGE_MANAGER=apt
TERMUX_APP_PID=15008
TERMUX_APP__DATA_DIR=/data/user/0/com.termux
TERMUX_APP__LEGACY_DATA_DIR=/data/data/com.termux
TERMUX_APP__SE_FILE_CONTEXT=u:object_r:app_data_file:s0:c124,c256,c512,c768
TERMUX_APP__SE_INFO=default:targetSdkVersion=28:complete
TERMUX_IS_DEBUGGABLE_BUILD=0
TERMUX_MAIN_PACKAGE_FORMAT=debian
TERMUX_VERSION=0.118.3
TERMUX__HOME=/data/data/com.termux/files/home
TERMUX__PREFIX=/data/data/com.termux/files/usr
TERMUX__ROOTFS_DIR=/data/data/com.termux/files
TERMUX__SE_PROCESS_CONTEXT=u:r:untrusted_app:s0:c124,c256,c512,c768
TERMUX__USER_ID=0
Packages CPU architecture:
aarch64
Subscribed repositories:
# sources.list
deb https://packages-cf.termux.dev/apt/termux-main stable main
Updatable packages:
command-not-found/stable 2.4.0-78 aarch64 [upgradable from: 2.4.0-68]
curl/stable 8.14.1 aarch64 [upgradable from: 8.12.1]
debianutils/stable 5.22 aarch64 [upgradable from: 5.21]
diffutils/stable 3.12 aarc

In [6]:
! inxi

CPU: 2x 4-core AArch64 (-MCP AMP-) speed/min/max: 332/499:403/1651:1210 MHz
Kernel: 3.10.108-lk.r17_rev aarch64 Up: 4d 3h 14m Mem: 1.1/1.8 GiB (61.2%)
Storage: 14.56 GiB (235.2% used) Procs: 9 Shell: python3.12 inxi: 3.3.38


In [7]:
! lscpu

Architecture:            aarch64
  CPU op-mode(s):        32-bit, 64-bit
  Byte Order:            Little Endian
CPU(s):                  8
  On-line CPU(s) list:   0,2,4-7
  Off-line CPU(s) list:  1,3
Vendor ID:               ARM
  Model name:            Cortex-A53
    Model:               4
    Thread(s) per core:  1
    Core(s) per cluster: 3
    Socket(s):           -
    Cluster(s):          2
    Stepping:            r0p4
    CPU(s) scaling MHz:  61%
    CPU max MHz:         1651.2000
    CPU min MHz:         0.0000
    Flags:               fp asimd evtstrm aes pmull sha1 sha2 crc32


In [1]:
! clang --version

clang version 20.1.6
Target: aarch64-unknown-linux-android24
Thread model: posix
InstalledDir: /data/data/com.termux/files/usr/bin


In [2]:
! as --version

GNU assembler (GNU Binutils) 2.44
Copyright (C) 2025 Free Software Foundation, Inc.
This program is free software; you may redistribute it under the terms of
the GNU General Public License version 3 or later.
This program has absolutely no warranty.
This assembler was configured for a target of `aarch64-linux-android'.


In [3]:
! ld --version

GNU ld (GNU Binutils) 2.44
Copyright (C) 2025 Free Software Foundation, Inc.
This program is free software; you may redistribute it under the terms of
the GNU General Public License version 3 or (at your option) a later version.
This program has absolutely no warranty.


## tiny.c

In [4]:
%%writefile tiny1.c
int main(void) {
    return 42; 
}

Writing tiny1.c


In [5]:
! clang -Oz -s tiny1.c

In [6]:
! ./a.out ; echo $?

42


In [7]:
! wc -c a.out

4144 a.out


In [8]:
! size a.out

   text	   data	    bss	    dec	    hex	filename
   1195	    464	   1824	   3483	    d9b	a.out


In [9]:
! objdump -h a.out


a.out:     file format elf64-littleaarch64

Sections:
Idx Name          Size      VMA               LMA               File off  Algn
  0 .interp       00000015  0000000000000270  0000000000000270  00000270  2**0
                  CONTENTS, ALLOC, LOAD, READONLY, DATA
  1 .note.android.ident 00000098  0000000000000288  0000000000000288  00000288  2**2
                  CONTENTS, ALLOC, LOAD, READONLY, DATA
  2 .dynsym       00000060  0000000000000320  0000000000000320  00000320  2**3
                  CONTENTS, ALLOC, LOAD, READONLY, DATA
  3 .gnu.version  00000008  0000000000000380  0000000000000380  00000380  2**1
                  CONTENTS, ALLOC, LOAD, READONLY, DATA
  4 .gnu.version_r 00000020  0000000000000388  0000000000000388  00000388  2**2
                  CONTENTS, ALLOC, LOAD, READONLY, DATA
  5 .gnu.hash     0000001c  00000000000003a8  00000000000003a8  000003a8  2**3
                  CONTENTS, ALLOC, LOAD, READONLY, DATA
  6 .dynstr       00000066  00000000000003c4  000

In [10]:
%%writefile tiny.c
extern void _exit(int);

void _start(void) {
    _exit(42); 
}

Writing tiny.c


In [11]:
%%writefile tiny.asm
    .global _start
    .text
_start:
    /* syscall exit (int status) */
    mov    w8, #93  /* exit is syscall #1 */
    mov    x0, #42  /* status  := 42 */
    svc    #0

Writing tiny.asm


In [12]:
! as tiny.asm -o tiny.o

\
Linking with clang:

In [13]:
! clang -Wall -s -nostartfiles -nostdlib tiny.o

In [14]:
! ./a.out ; echo $?

42


In [15]:
! wc -c a.out

1688 a.out


In [16]:
! size a.out

   text	   data	    bss	    dec	    hex	filename
    122	    160	   3296	   3578	    dfa	a.out


In [18]:
! objdump -h a.out


a.out:     file format elf64-littleaarch64

Sections:
Idx Name          Size      VMA               LMA               File off  Algn
  0 .interp       00000015  0000000000000200  0000000000000200  00000200  2**0
                  CONTENTS, ALLOC, LOAD, READONLY, DATA
  1 .dynsym       00000018  0000000000000218  0000000000000218  00000218  2**3
                  CONTENTS, ALLOC, LOAD, READONLY, DATA
  2 .gnu.hash     0000001c  0000000000000230  0000000000000230  00000230  2**3
                  CONTENTS, ALLOC, LOAD, READONLY, DATA
  3 .dynstr       00000025  000000000000024c  000000000000024c  0000024c  2**0
                  CONTENTS, ALLOC, LOAD, READONLY, DATA
  4 .text         0000000c  0000000000000274  0000000000000274  00000274  2**2
                  CONTENTS, ALLOC, LOAD, READONLY, CODE
  5 .dynamic      000000a0  0000000000004280  0000000000004280  00000280  2**3
                  CONTENTS, ALLOC, LOAD, DATA
  6 .relro_padding 00000ce0  0000000000004320  0000000000004320  0

In [19]:
! strip --strip-all --strip-section-headers a.out

In [20]:
! objdump -h a.out


a.out:     file format elf64-littleaarch64

Sections:
Idx Name          Size      VMA               LMA               File off  Algn


In [21]:
! wc -c a.out

800 a.out


\
Linking and gerenating executable direct from assembly and linker

In [36]:
! rm a.out

In [37]:
! ls -lh tiny.o

-rw-------. 1 u0_a124 u0_a124 728 Jun  7 14:27 tiny.o


In [38]:
! ld --strip-all --gc-sections -static tiny.o

In [39]:
! ./a.out ; echo $?

42


In [40]:
! wc -c a.out

344 a.out


In [41]:
! strip --strip-all a.out

In [42]:
! wc -c a.out

344 a.out


In [43]:
! ./a.out ; echo $?

42


In [44]:
! objdump -h a.out


a.out:     file format elf64-littleaarch64

Sections:
Idx Name          Size      VMA               LMA               File off  Algn
  0 .text         0000000c  0000000000400078  0000000000400078  00000078  2**2
                  CONTENTS, ALLOC, LOAD, READONLY, CODE


In [45]:
! as -al tiny.asm

AARCH64 GAS  tiny.asm 			page 1


   1              	    .global _start
   2              	    .text
   3              	_start:
   4              	    /* syscall exit (int status) */
   5 0000 A80B8052 	    mov    w8, #93  /* exit is syscall #1 */
   6 0004 400580D2 	    mov    x0, #42  /* status  := 42 */
   7 0008 010000D4 	    svc    #0


In [46]:
! objdump -d a.out


a.out:     file format elf64-littleaarch64


Disassembly of section .text:

0000000000000000 <_start>:
   0:	52800ba8 	mov	w8, #0x5d                  	// #93
   4:	d2800540 	mov	x0, #0x2a                  	// #42
   8:	d4000001 	svc	#0x0


In [47]:
! hexdump -C a.out

00000000  7f 45 4c 46 02 01 01 00  00 00 00 00 00 00 00 00  |.ELF............|
00000010  01 00 b7 00 01 00 00 00  00 00 00 00 00 00 00 00  |................|
00000020  00 00 00 00 00 00 00 00  18 01 00 00 00 00 00 00  |................|
00000030  00 00 00 00 40 00 00 00  00 00 40 00 07 00 06 00  |....@.....@.....|
00000040  a8 0b 80 52 40 05 80 d2  01 00 00 d4 00 00 00 00  |...R@...........|
00000050  00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  |................|
00000060  00 00 00 00 00 00 00 00  00 00 00 00 03 00 01 00  |................|
00000070  00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  |................|
00000080  00 00 00 00 03 00 02 00  00 00 00 00 00 00 00 00  |................|
00000090  00 00 00 00 00 00 00 00  00 00 00 00 03 00 03 00  |................|
000000a0  00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  |................|
000000b0  01 00 00 00 00 00 01 00  00 00 00 00 00 00 00 00  |................|
000000c0  00 00 00 00 00 00 00 00  04 00 00 00 10 00

In [48]:
! size a.out

   text	   data	    bss	    dec	    hex	filename
     12	      0	      0	     12	      c	a.out


In [49]:
! size -Ax a.out

a.out  :
section   size   addr
.text    0xc    0x0
.data    0x0    0x0
.bss     0x0    0x0
Total    0xc




# C with more aggressive options

In [70]:
%%writefile tiny2.c
extern void _exit(int);

void _start(void) {
    _exit(42); 
}

Overwriting tiny2.c


In [71]:
%%writefile tiny2.asm
    .arch armv8-a
    .section .text
    .globl _exit
    .type _exit, %function
_exit:
    /* syscall exit (int status) */
    mov    w8, #93  /* exit is syscall #1 */
    mov    x0, #42  /* status  := 42 */
    svc    #0

Overwriting tiny2.asm


In [72]:
! as tiny2.asm -o tiny2.o

- -ffreestanding is used in embedded systems where there is no operating system, and everything has to be self-contained.
    - void main(void) .
    - The standard header files <stdio.h>, <string.h> and so on, are not to be used.
- On ARM64 the "-Oz -ffreestanding" generated assembly does not include SP stack initialization.
- -fident and -fno-ident control whether the output file contains the compiler name and version information.
- -nostdlib implies the individual options -nodefaultlibs and -nostartfiles .
    - the only libraries linked are exactly those that you explicitly name to the linker using the -l flag .
    - http://cs107e.github.io/guides/gcc/

In [73]:
%%bash
clang -Wall -Oz -static \
      -fno-inline -s -g0 -DNDEBUG \
      -nostartfiles -nostdlib -ffreestanding \
      -flto \
      -fno-stack-protector -fno-exceptions \
      -fno-ident -fno-unwind-tables -fno-asynchronous-unwind-tables \
      -ffunction-sections -fdata-sections \
      -fno-plt -fno-pic -fno-pie \
      -fmerge-all-constants -fno-common \
      -fomit-frame-pointer -fno-builtin \
      -Wl,--gc-sections,--strip-all,--strip-debug \
      -Wl,--hash-style=sysv,--build-id=none \
      -Wl,--no-eh-frame-hdr,--no-gnu-unique,--omagic,--nmagic,--no-relax \
      -Wl,-z,norelro,-z,noseparate-code,-z,max-page-size=1,-z,common-page-size=1 \
      tiny2.o tiny2.c -o a.out
strip --strip-all --strip-debug --strip-unneeded --strip-section-headers \
      -R .comment \
      --remove-section=.note.gnu.build-id \
      --remove-section=.note \
      --remove-section=.note.ABI-tag \
      --remove-section=.gnu.hash \
      --remove-section=.dynsym \
      --remove-section=.dynstr \
      --remove-section=.gnu.version \
      --remove-section=.gnu.version_r \
      --remove-section=.rel.dyn \
      --remove-section=.rel.plt \
      --remove-section=.init \
      --remove-section=.plt \
      --remove-section=.fini \
      --remove-section=.rodata \
      --remove-section=.eh_frame_hdr \
      --remove-section=.eh_frame \
      --remove-section=.init_array \
      --remove-section=.fini_array \
      --remove-section=.jcr \
      --remove-section=.got \
      --remove-section=.got.plt \
      --remove-section=.data \
      --remove-section=.bss \
      --remove-section=.comment \
      --remove-section=.strtab \
      --remove-section=.symtab \
      a.out

ld.lld: warning: -z max-page-size set, but paging disabled by omagic or nmagic
ld.lld: warning: -z common-page-size set, but paging disabled by omagic or nmagic


In [84]:
! ./a.out ; echo $?

42


In [85]:
! wc -c a.out

196 a.out


In [86]:
! readelf -S a.out


There are no sections in this file.


In [87]:
! readelf -s a.out


Dynamic symbol information is not available for displaying symbols.


In [88]:
! ls -lh a.out

-rwx------. 1 u0_a124 u0_a124 196 Jun  7 14:48 a.out


In [91]:
! size a.out

   text	   data	    bss	    dec	    hex	filename
      0	      0	      0	      0	      0	a.out


In [92]:
! objdump -h a.out


a.out:     file format elf64-littleaarch64

Sections:
Idx Name          Size      VMA               LMA               File off  Algn


In [93]:
! objdump -Dz a.out


a.out:     file format elf64-littleaarch64



In [94]:
! objdump -d a.out


a.out:     file format elf64-littleaarch64



In [95]:
! hexdump -C a.out

00000000  7f 45 4c 46 02 01 01 00  00 00 00 00 00 00 00 00  |.ELF............|
00000010  02 00 b7 00 01 00 00 00  bc 00 20 00 00 00 00 00  |.......... .....|
00000020  40 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  |@...............|
00000030  00 00 00 00 40 00 38 00  02 00 00 00 00 00 00 00  |....@.8.........|
00000040  01 00 00 00 07 00 00 00  b0 00 00 00 00 00 00 00  |................|
00000050  b0 00 20 00 00 00 00 00  b0 00 20 00 00 00 00 00  |.. ....... .....|
00000060  14 00 00 00 00 00 00 00  14 00 00 00 00 00 00 00  |................|
00000070  04 00 00 00 00 00 00 00  51 e5 74 64 06 00 00 00  |........Q.td....|
00000080  00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  |................|
*
000000b0  a8 0b 80 52 40 05 80 d2  01 00 00 d4 40 05 80 52  |...R@.......@..R|
000000c0  fc ff ff 17                                       |....|
000000c4


---

## Minimal command line equivalent

In [101]:
%%bash
clang -Oz -static -nostdlib -nostartfiles -ffreestanding \
  -fno-builtin -fno-stack-protector -fno-exceptions \
  -ffunction-sections -fdata-sections \
  -Wl,--gc-sections,--strip-all,--build-id=none \
  -Wl,-z,notext,-z,norelro,-z,noseparate-code,--omagic,--nmagic \
  tiny2.o tiny2.c -o a.out
strip --strip-all --strip-section-headers a.out

ld.lld: warning: -z max-page-size set, but paging disabled by omagic or nmagic


In [102]:
! wc -c a.out

196 a.out


In [103]:
! size a.out

   text	   data	    bss	    dec	    hex	filename
      0	      0	      0	      0	      0	a.out


In [104]:
! ./a.out ; echo $?

42


---

- -Wl,–gc-sections: Tell the linker to garbage collect and discard unused sections
- -Wl,-static: Link against static libraries. Required for dead-code elimination

In [105]:
%%bash
clang tiny2.o tiny2.c \
-Oz -s -ffreestanding -nostartfiles -nostdlib -static -Wl,--gc-sections

In [106]:
! ls -lh a.out

-rwx------. 1 u0_a124 u0_a124 576 Jun  7 14:51 a.out


In [107]:
! ./a.out ; echo $?

42


In [108]:
! size a.out

   text	   data	    bss	    dec	    hex	filename
     20	      0	      0	     20	     14	a.out


In [109]:
! objdump -d a.out


a.out:     file format elf64-littleaarch64


Disassembly of section .text:

00000000002000e8 <.text>:
  2000e8:	52800ba8 	mov	w8, #0x5d                  	// #93
  2000ec:	d2800540 	mov	x0, #0x2a                  	// #42
  2000f0:	d4000001 	svc	#0x0
  2000f4:	52800540 	mov	w0, #0x2a                  	// #42
  2000f8:	17fffffc 	b	0x2000e8


In [110]:
! objdump -h a.out


a.out:     file format elf64-littleaarch64

Sections:
Idx Name          Size      VMA               LMA               File off  Algn
  0 .text         00000014  00000000002000e8  00000000002000e8  000000e8  2**2
                  CONTENTS, ALLOC, LOAD, READONLY, CODE
  1 .comment      00000029  0000000000000000  0000000000000000  000000fc  2**0
                  CONTENTS, READONLY


In [111]:
! objdump -s a.out


a.out:     file format elf64-littleaarch64

Contents of section .text:
 2000e8 a80b8052 400580d2 010000d4 40058052  ...R@.......@..R
 2000f8 fcffff17                             ....            
Contents of section .comment:
 0000 004c696e 6b65723a 204c4c44 2032302e  .Linker: LLD 20.
 0010 312e3600 636c616e 67207665 7273696f  1.6.clang versio
 0020 6e203230 2e312e36 00                 n 20.1.6.       


In [112]:
! strip --strip-all --strip-section-headers a.out

In [113]:
! ./a.out ; echo $?

42


In [114]:
! ls -lh a.out

-rwx------. 1 u0_a124 u0_a124 252 Jun  7 14:51 a.out


In [115]:
! objdump -h a.out


a.out:     file format elf64-littleaarch64

Sections:
Idx Name          Size      VMA               LMA               File off  Algn


In [116]:
! objdump -s a.out


a.out:     file format elf64-littleaarch64



In [117]:
! hexdump -C a.out

00000000  7f 45 4c 46 02 01 01 00  00 00 00 00 00 00 00 00  |.ELF............|
00000010  02 00 b7 00 01 00 00 00  f4 00 20 00 00 00 00 00  |.......... .....|
00000020  40 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  |@...............|
00000030  00 00 00 00 40 00 38 00  03 00 00 00 00 00 00 00  |....@.8.........|
00000040  06 00 00 00 04 00 00 00  40 00 00 00 00 00 00 00  |........@.......|
00000050  40 00 20 00 00 00 00 00  40 00 20 00 00 00 00 00  |@. .....@. .....|
00000060  a8 00 00 00 00 00 00 00  a8 00 00 00 00 00 00 00  |................|
00000070  08 00 00 00 00 00 00 00  01 00 00 00 05 00 00 00  |................|
00000080  00 00 00 00 00 00 00 00  00 00 20 00 00 00 00 00  |.......... .....|
00000090  00 00 20 00 00 00 00 00  fc 00 00 00 00 00 00 00  |.. .............|
000000a0  fc 00 00 00 00 00 00 00  00 40 00 00 00 00 00 00  |.........@......|
000000b0  51 e5 74 64 06 00 00 00  00 00 00 00 00 00 00 00  |Q.td............|
000000c0  00 00 00 00 00 00 00 00  00 00 00 00 00 00

---

## Hello World

In [118]:
%%writefile tiny3.c
#include <stdio.h>

int main(void) {
    puts("Hello World!\n");
}

Writing tiny3.c


In [119]:
! clang -Wall tiny3.c

In [120]:
! ./a.out

Hello World!



In [121]:
! ls -lh a.out

-rwx------. 1 u0_a124 u0_a124 5.7K Jun  7 14:52 a.out


---

In [123]:
%%writefile tiny4.c
#include <stdio.h>

void _start(void) {
    puts("Hello World!\n");
}

Overwriting tiny4.c


In [154]:
%%bash
clang tiny4.c \
    -Wall -Oz -s -nostartfiles -fno-exceptions \
    -fno-ident -fno-asynchronous-unwind-tables  \
    -ffunction-sections -fdata-sections -Wl,--gc-sections,--strip-all \
    -Wl,--gc-sections,--strip-all,--build-id=none

In [155]:
! ./a.out

Hello World!



In [156]:
! wc -c a.out

2296 a.out


In [157]:
! strip --strip-all --strip-section-headers a.out

In [158]:
! ./a.out

Hello World!



In [159]:
! wc -c a.out

1104 a.out


In [162]:
! objdump -h a.out


a.out:     file format elf64-littleaarch64

Sections:
Idx Name          Size      VMA               LMA               File off  Algn


In [160]:
! objdump -Dz a.out


a.out:     file format elf64-littleaarch64



In [163]:
! hexdump -C a.out

00000000  7f 45 4c 46 02 01 01 00  00 00 00 00 00 00 00 00  |.ELF............|
00000010  03 00 b7 00 01 00 00 00  c0 02 00 00 00 00 00 00  |................|
00000020  40 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  |@...............|
00000030  00 00 00 00 40 00 38 00  07 00 00 00 00 00 00 00  |....@.8.........|
00000040  06 00 00 00 04 00 00 00  40 00 00 00 00 00 00 00  |........@.......|
00000050  40 00 00 00 00 00 00 00  40 00 00 00 00 00 00 00  |@.......@.......|
00000060  88 01 00 00 00 00 00 00  88 01 00 00 00 00 00 00  |................|
00000070  08 00 00 00 00 00 00 00  03 00 00 00 04 00 00 00  |................|
00000080  c8 01 00 00 00 00 00 00  c8 01 00 00 00 00 00 00  |................|
00000090  c8 01 00 00 00 00 00 00  15 00 00 00 00 00 00 00  |................|
000000a0  15 00 00 00 00 00 00 00  01 00 00 00 00 00 00 00  |................|
000000b0  01 00 00 00 05 00 00 00  00 00 00 00 00 00 00 00  |................|
000000c0  00 00 00 00 00 00 00 00  00 00 00 00 00 00

---

In [171]:
%%writefile tiny5.c
#include <stdio.h>
#include <unistd.h>
#include <string.h>

void _start(void) {
    char *s="Hello World!\n";
    write(1,s,strlen(s));
}

Overwriting tiny5.c


In [172]:
%%bash
clang tiny5.c \
    -Oz -s -nostartfiles \
    -fno-ident -fno-asynchronous-unwind-tables  \
    -ffunction-sections -fdata-sections -Wl,--gc-sections,--strip-all

In [173]:
! ./a.out

Hello World!


In [174]:
! wc -c a.out

2312 a.out


In [175]:
! strip --strip-all --strip-section-headers a.out

In [176]:
! ./a.out

Hello World!


In [177]:
! wc -c a.out

1120 a.out
